# 连接网络

In [ ]:
#-*-coding:utf-8-*-
import json
import networkx as nx
from tqdm import tqdm
import numpy as np
import os

###设置工作路径
os.chdir('/home/dell-none/jupyter_notebook_code/covid19')

###读入前述存储两两序列间转换概率的json文件
with open('prob_cds_results_l.json','r')as f3:
    P=json.load(f3)
a=np.array(P)

min_pro=np.percentile(a,10)###阈值10%，大于10%的边都保留
print(min_pro)


###创建空网络

g1=nx.Graph()

###读入需建立网络的序列
dic1 = get_dic('cds2_l.fasta')
keys = list(dic1.keys())###序列名为节点 
g1.add_nodes_from(keys)

###遍历两两序列节点间概率
for i in P:
#     plst=[ii[2] for ii in i if ii[2]!=1]
#     max_v=sorted(plst,reverse=True)[:1]
#     print(max_v)
    for ii in i :
#         if ii[2] in max_v:
#             g1.add_edge(ii[0],ii[1],weight=ii[2])
        try:
            if ii[2]>=min_pro: ###大于阈值就两者之间加边
                g1.add_edge(ii[0],ii[1],weight=ii[2])
        except:
            pass

print(g1.number_of_nodes())
print(g1.number_of_edges())

print(nx.number_connected_components(g1))


In [ ]:
###转换为数组格式
keys=np.array(keys)
lst2=np.array(P)
# del P

###寻找子网络间概率最大的两个节点
def max_pro(com_a, com_b):
    prob = []
#     print(com_a)
    for a in com_a:
#         print(a)
        for b in com_b:
#             print(a)
#             print(b)
            index1 = np.argwhere(keys == a)
            index2 = np.argwhere(keys == b)
#             print(index1)
#             print(index2)
            try:
                pro = float(lst2[index1, index2 - 1][0][0][2])
                
#                 print(pro)
                
                prob.append((a, b, pro))
            except:
                prob.append(('unac', 'unac', 0))
#             print(prob)
    maxp = max([i[2] for i in prob])

    if maxp != 0:
        return [i for i in prob if i[2] == maxp]
    else:
        return []


# #-----------------------------------------------------------------------------

##重复上述过程，在子网络间概率最大的两节点之间添加边
def process(k):
    p_lst = []
    length = Length[:k] + Length[k + 1:]
    for j in length:

        com_a = list_com[k]
#         print(com_a)
        com_b = list_com[j]
        dis = max_pro(com_a, com_b)

        p_lst.extend(dis)

#     print(p_lst)
    max_p = max([i[2] for i in p_lst])
#     print(i[2])
#     print(type(i[2]))
    
    new_edge = [i for i in p_lst if i[2] == max_p]
    return new_edge

###循环，直至连成一个连通的网络
while nx.number_connected_components(g1)>1:

    alist = []

    list_com = [list(i) for i in list(nx.connected_components(g1))]
    
    Length = list(range(len(list_com)))
    
    
    if __name__ == '__main__':
        threads = 10
        with Pool(threads) as p:
            alist = list(
                tqdm(p.imap(process, Length), total=len(Length), desc="creating new edges")
            )
        del lst2
        for edge in alist:
#             print(edge)
            g1.add_weighted_edges_from(edge)

###查看最终连通的网络参数
print(nx.number_connected_components(g1))
nx.write_gpickle(g1, 'net_f_test.gpickle')

print(g1.number_of_nodes())
print(g1.number_of_edges())

print(nx.number_connected_components(g1))

# 无向网络转换成有向网络

In [ ]:
####确定每个节点的日期大小

df1=pd.read_excel('sample_info(l).xlsx')###sample_seq的信息路径
tlist=df1['collection_time'].values.tolist()
elist=df1['EPI_info'].values.tolist()

###提取时间
date_dic={}
for i in range(len(tlist)):
    key='>'+elist[i]
    
    date_dic[key]=tlist[i]
with open('epi_date.json','w')as f2:
    json.dump(date_dic,f2)

###时间转换成数字型 
def get_date(time):
    date=time.split('-')
    if len(date)==3:
        info=int(date[0])+float(int(date[1])/13)+float(int(date[2])/366)
    elif len(date)==2:
        info = int(date[0]) + float(int(date[1]) / 13)
    else:
        info=int(date[0] )
    return info
dic_date={}
for k,v in date_dic.items():
    date=get_date(v)
    dic_date[k]=v

###时间转换成数字型存入json文件
with open('seq_date.json','w')as f3:
    json.dump(dic_date,f3)

ID_Date={}
with open('seq_date.json','r')as f2:
    ID_Date=json.load(f2)

In [ ]:
###读入原始连通的无向网络
g1=nx.read_gpickle('net_f_test.gpickle')
print(nx.number_of_nodes(g1))
print(nx.number_connected_components(g1))


dic_pro={}

n=0

for lst1 in tqdm(P):
    dic1={}

    for i in lst1:
        dic1[i[1]]=i[2]

    try:
        dic_pro[keys[n]]=dic1
        n += 1
    except:
        pass
    
def get_start(root,g):
    nodes=[]
    for i in g.nodes():
        nodes.append([i, dic_pro[root][i], ID_Date[i]])
    lst=sorted(nodes,key=itemgetter(1,2))
    return lst[-1][0]

def DirectedNet(G, start, Matrix_Dict):
    Matrix_Dict[start][start]=1
    short_path = {}
    for i in G.nodes():
        if i!=start:
            path = nx.shortest_path_length(G, start, i, weight='weight')
            short_path[i]=path
            # print(path)
        else:
            short_path[i]=0
    edges = []
    for i in G.edges():
        edges.append(
            tuple(sorted(list(i), key=lambda x: (short_path[x], Matrix_Dict[start][x], ID_Date[x])) + [2 ** (-Matrix_Dict[i[0]][i[1]])]))
        #一对边的两个节点排序，依据为与起点间的路径，概率，时间，列表添加排序结果和两点概率负对数
    p = []
    for i in G.nodes():
        if i!=start:
            p.append(nx.shortest_path(
            G, source=start, target=i, weight='weight'))#最短路径列表

    Paths = []
    for line in p:
        for i in range(len(line) - 1):
            A, B = line[i], line[i + 1]
            Paths.append((A, B, 2**(-Matrix_Dict[A][B])))#每条路径内两点间概率负对数

    Paths = sorted(set(Paths), key=lambda x: Paths.index(x))
    G = nx.DiGraph()
    G.add_weighted_edges_from(Paths)
    aaa = []
    for i in Paths:
        if i in edges:
            aaa.append(i)
    for i in aaa:
        Paths.remove(i)
        edges.remove(i)
    Paths = [(i[1], i[0], i[2]) for i in Paths]
    for i in Paths:
        if i in edges:
            edges.remove(i)
#以上几步，去除骨架以外的没有方向的边，由下面的循环重新定方向
    for i in edges:
        G.add_weighted_edges_from([i])

    try:
        nx.find_cycle(G)
        while nx.find_cycle(G):
            a = nx.find_cycle(G)
            G.remove_edge(a[-1][0], a[-1][1])
            G.add_edge(a[-1][1], a[-1][0],
                       weight=Matrix_Dict[a[-1][1]][a[-1][0]])
            try:
                nx.find_cycle(G)
                G.remove_edge(a[-1][1], a[-1][0])
                pass
            except:
                break
    except:
        print('No cycle')

        pass

    print('Directed:')
    print(G.number_of_nodes())
    print(G.number_of_edges())
    return G

root='>EPI_ISL_435070'###设置root节点，从该节点开始确定网络方向，此处 root为wuhan01
g2=nx.DiGraph()

H = list(g1.subgraph(c) for c in nx.connected_components(g1))
for i in H:

    if root in i.nodes():
        g=DirectedNet(g1,root,dic_pro)

        g2.add_weighted_edges_from(g.edges.data(data='weight'))
    else:
        start=get_start(root,g1)

        print(start)
        g = DirectedNet(g1, start, dic_pro)
        g2.add_weighted_edges_from(g.edges.data(data='weight'))

###最终有向网络写入文件        
nx.write_gpickle(g2, 'sars-cov-2.gpickle')
nx.write_graphml(g2,'cds_sars-cov-2_net.graphml')

# 随机游走确定路径

In [ ]:
###读取建立的有向网络
g1= nx.DiGraph()
net=pickle5.load(open('final_di_net_l.gpickle','rb'))
if __name__ == '__main__':
    g1.add_weighted_edges_from(net.edges(data='weight'))
print(g1.number_of_nodes())

with open('epi_date.json', 'r')as f1:
    dic_date=json.load(f1)
with open('seq_date.json', 'r')as f2:
    date_num=json.load(f2)

###日期拆分   
def get_date(node):
    
    da_ls=dic_date[node].split('/')

    return da_ls

###时间较早的节点做起点
star_nodes=[]
for node in g1_direc.nodes():
    da_ls=get_date(node)
    if (int(da_ls[0])==2019) or (int(da_ls[0])==2020 and int(da_ls[1])<4):
        star_nodes.append(node)
# print(len(star_nodes))

###定义随机游走
def deepwalk_walk(g,start_node):
    walk = [start_node]

    while abs(date_num[walk[0]]-date_num[walk[-1]])<5:###此处可给出限制节点的条件，例如采样时间超过多久的不算在内
        cur = walk[-1]

        cur_nbrs = list(g.successors(cur))

        if len(cur_nbrs) > 0:
            walk.append(np.random.choice(cur_nbrs))
        else:
            break

    return walk

def _simulate_walks(num_walks, nodes):
    walks = []
    for i in tqdm(range(num_walks)):
        v=random.choice(nodes)
        walks.append(deepwalk_walk(g1_direc, start_node=v))
    return walks
###网络上从起点开始实施随机游走
walks=_simulate_walks(20000,star_nodes)

###收集路径上相邻超过一个月的点
lst_node=[]

for i in walks:
    if len(i)>=2:
        for j in range(len(i)-1):
            for k in range(j+1,len(i)):
                if abs(date_num[i[j]]-date_num[i[k]])>0.08:###此处设置为 超过一个月的值
                    lst_node.append((i[j],i[k]))
###节点对list
lst_node=list(set(lst_node))

###按照节点对整理序列
dic1=get_dic('seq.fasta')

def get_seq(node):
    seq=dic1[node]
    return seq

###取出节点对序列
def check_seq(node1,node2):
    dif_pos=[]
    s1,s2=get_seq(node1),get_seq(node2)
    for i in range(len(s1)):
        if s1[i]!= s2[i]:
            dif_pos.append(i)
    return dif_pos

tra_dic={};dic_lst=[]
for i in tqdm(lst_node):
    tra_dic[i]=check_seq(i[0],i[1])

    if len(tra_dic)>5000:
        dic_lst.append(tra_dic)
        tra_dic={}
###序列滑窗        
def inter(dic):
    new={};sl_ls=[(i,i+600)  for i in range(0,29118,3)]
    for k,v in dic.items():
        
        b=[]
        for i in v:
            for j in sl_ls:
                if i >= j[0] and i<=j[1]:
                    b.append(j)
        new[k]=b
    return new 

#序列滑窗改写并行
def cal_(dic_lst):
    with Pool(10) as pool:
        res=pool.map(inter,dic_lst)
        pool.close()
        pool.join()
    return list(res)
result=cal_(dic_lst)
print(len(result))


###节点对滑窗序列改写成axt格式，该格式是kaks_calculator所需要的格式
def tran_fas(dic,dic1):
    new_={}
    
    for k,v in dic.items():
        for i in v:
            key=k[0]+','+k[1]+'('+str(i[0])+')'
            seq1,seq2=dic1[k[0]][i[0]:i[1]],dic1[k[1]][i[0]:i[1]]
            new_[key]=seq1+'\n'+seq2+'\n'
    return new_

arg_l=[(i,dic1) for i in result]

#改写axt格式，改写并行
def cal2(arg_l):
    with Pool(10) as pool:
        res=pool.starmap(tran_fas,arg_l)
        pool.close()
        pool.join()
    return list(res)    
res2=cal2(arg_l)
print(len(res2))

# del arg_l
del dic1

###节点对序列写入文件
n=0
for i in res2:
    filename='fas/fas_%s.axt'%n

    with open(filename,'w')as f2:
        for k,v in i.items():
            f2.write(k+'\n'+v+'\n')
#             del i
    n+=1

# 估算SER

In [ ]:
#-*-coding:utf-8-*-
import json
import numpy as np
import random
from tqdm import tqdm
import math 
import re
import pandas as pd
import pickle
import pickle5
import networkx as nx
import os 
import datetime
from multiprocessing import Pool
from matplotlib import pyplot as plt

###设置工作路径
os.chdir('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt4/')

###调用计算ks
def fun(file,path):
    os.system('/home/dell-none/Desktop/KaKs_Calculator2.0/bin/Linux/KaKs_Calculator -i %s -o %s -m MLWL'%(file,path))

###改写函数并行计算
def main():
        pool = Pool(11)
        n=0
        for n in range(num_file):### 此处括号内是生成的axt文件数
            filename='fas_%s.axt'%(n)
            filepath='fas_%s.kaks'%(n)
            pool.apply_async(fun, args=(filename,filepath))

        pool.close()
        pool.join()
if __name__ == '__main__':
    main()
    
os.system("rm fas_*.axt")



In [ ]:
###转换成csv文件
def get_data(fileneme):
    with open(fileneme,'r')as f1:
        lines=f1.readlines()
    if lines:
        titles=[]
        data=[]
        for line in lines:
            if 'Sequence' in line:
                titles=line.split()[:]
#                 print(len(titles))
            else:
                lst=line.split()[:]
                data.append(lst)
        df=pd.DataFrame(data,columns=titles)
        df.Ks[df['Ks']=='NA'] = 0
        return df
	
def pro_csv(df,filepath):
    
    df.to_csv(filepath,index=False)
###读取csv   
def get_df(filename):
    df = pd.read_csv(filename)
    return df



###获取序列收集时间相差天数
with open('seq_date.json','r')as f1:
    date_dic=json.load(f1)

def daytime(d1,d2):
    tlist1=d1.split("/");tlist2=d2.split("/")

    if len(tlist1)!=2 and len(tlist2)!=2:
        day1 = datetime.datetime(int(d1.split("/")[0]),int(d1.split("/")[1]),int(d1.split("/")[2]))
        day2 = datetime.datetime(int(d2.split("/")[0]),int(d2.split("/")[1]),int(d2.split("/")[2]))
        interval = day2 - day1
        time=interval.days
    if len(tlist1)==2 and len(tlist2)!=2:
        day1 = datetime.datetime(int(d1.split("/")[0]),int(d1.split("/")[1]),15)
        day2 = datetime.datetime(int(d2.split("/")[0]),int(d2.split("/")[1]),int(d2.split("/")[2]))
        interval = day2 - day1
        time=interval.days
    if len(tlist1)!=2 and len(tlist2)==2:
        day2 = datetime.datetime(int(d2.split("/")[0]),int(d2.split("/")[1]),15)
        day1 = datetime.datetime(int(d1.split("/")[0]),int(d1.split("/")[1]),int(d1.split("/")[2]))
        interval = day2 - day1
        time=interval.days
    if len(tlist1)==2 and len(tlist2)==2:
        day2 = datetime.datetime(int(d2.split("/")[0]),int(d2.split("/")[1]),15)
        day1 = datetime.datetime(int(d1.split("/")[0]),int(d1.split("/")[1]),15)
        interval = day2 - day1
        time=interval.days


    return abs(time)


###添加收集时间相差天数信息
def add_day(df):
    day=[]
    seq=df['Sequence'].to_list()
    for i in seq: 
        time1=date_dic[i.split(',')[0]]
        time2=date_dic[i.split(',')[1].split('(')[0]]
        day.append(int(daytime(time1,time2)))
    return day

###转换成csv格式，并保留有用列信息
csv_lst=[]
for i in range(0,nn+1):
    filename1='%d.kaks'%(i)
    df=get_data(filename1)
    day=add_day(df)
    df['Time interval']=day

    df=df[['Sequence','Ks','S-Sites','S-Substitutions','Time interval']]
    df['Ks']=[float(i) for i in list(df['Ks'])]
    csv_lst.append(df)


for i in range(nn+1):
    os.remove('%d.kaks'%(i))



####合并dataframe
def merge_df(all_list): 
    fdf=pd.concat(all_list,axis=0)
    fdf['S-Substitutions'].fillna(0,inplace=True)
    return fdf

df1=merge_df(csv_lst)
print(df1.iloc[1,:])

###添加位置信息
position=[]
p1 = re.compile(r'[(](.*?)[)]', re.S) 
for i in df1['Sequence'].to_list():
    p=int(re.findall(p1, i)[0].split('-')[0])
    position.append(p)
    
df1['Position']=position
df1.to_csv('final_result.csv',index=False,sep=',')

##-----------------------------------------------------------

df=df1

del df1

###对每个位点线性拟合得到斜率，即为SER
rate_coef=[];pos=[];ci_pos=[]
from scipy import stats
from sklearn.linear_model import LinearRegression
grouped=df.groupby('Position')
for k,i in tqdm(grouped):
    
    pos1=i.reset_index(drop=True)
    
    x=np.array(pos1['Time interval']).reshape(pos1['Time interval'].shape[0],1)
    y=pos1['Ks']
    xl=np.array(pos1['Time interval'])
    model = LinearRegression(fit_intercept=False).fit(x, y)
    
    x_sum=np.sqrt(np.sum([(i-np.mean(xl))**2 for i in list(pos1['Time interval'])]))
    
    syx = np.sqrt(np.sum((np.array(y)-model.predict(x))**2)/(len(x)-2))###test时改了，应为x-2
    sb=syx/x_sum
    
    t = stats.t.ppf((1- 0.05/2), (len(x)-2))###test时改了，应为x-2
    ci=t*sb
    ci_pos.append(float(ci)*366)
    a = model.coef_
    pos.append(k)
    rate_coef.append(float(a*366))

df_pos=pd.DataFrame([pos,rate_coef,ci_pos]).T
df_pos.columns=['Position','SER','CI']

df_pos.to_csv('speci_pos_ser_fit.csv',index=False,sep=',')

# 位点添加CDS信息

In [ ]:
# os.chdir('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt')
df1=pd.read_csv('speci_pos_ser_fit.csv')
df1.head()


### create a list of our conditions，条件筛选,填入序列位点####位置需重新修改
conditions = [
    (df1['Position'] < 13203),
    (df1['Position'] >=13203) & (df1['Position'] < 21288),
    (df1['Position'] >=21288) & (df1['Position'] < 25107),
    (df1['Position'] >=25107) & (df1['Position'] < 25932),
    (df1['Position'] >=25932) & (df1['Position'] < 26157),
    (df1['Position'] >=26157) & (df1['Position'] < 26823),
    (df1['Position'] >=26823) & (df1['Position'] < 27006),
    (df1['Position'] >=27006) & (df1['Position'] < 27369),
    (df1['Position'] >=27369) & (df1['Position'] < 27498),
    (df1['Position'] >=27498) & (df1['Position'] < 27861),
    (df1['Position'] >=27861) & (df1['Position'] < 29118),
    ]

### create a list of the values we want to assign for each condition
values = ['ORF1a','ORF1b', 'S', 'ORF3a','E', 'M','ORF6','ORF7a','ORF7b','ORF8','N']

### create a new column and use np.select to assign values to it using our lists as arguments
df1['Gene'] = np.select(conditions, values)


###选取长度大于600bp的区域
df=df1[(df1['Gene']=='ORF1a')|(df1['Gene']=='ORF1b')|(df1['Gene']=='S')
       |(df1['Gene']=='ORF3a')|(df1['Gene']=='M')
       |(df1['Gene']=='N')].reset_index(drop=True)
df=df[['SER','Gene']]

###改成ncbi位置
cut_pos=[0,13202,21287,25106,25931,26156,26822,27005,27368,27497,27860]
values = ['ORF1a','ORF1b', 'S', 'ORF3a','E', 'M','ORF6','ORF7a','ORF7b','ORF8','N']
ncbi=[265,13466,21561,25391,26243,26521,27200,27392,27754,27892,28272]
###转换为NCBI对应的位置
def tran_pos(alst):
    new=[]
    
    
    for i in alst:
        num=i
        if num>27860:
            new.append((num-27860+28272))
        else:
            for j in range(len(cut_pos)-1):
                if cut_pos[j]<num<=cut_pos[j+1]:
                    new.append((num-cut_pos[j])+ncbi[j])

#     print(len(new))
    return new
df=tran_pos(list(df['Position']))
###df写入文件
df.to_csv('slope_all_SER.csv',index=False,sep=',')

# 各CDS统计学检验

In [ ]:
###计算中位数，四分位数，平均数
np.percentile(list(df['SER']),[50,25,75])
np.mean(list(df['SER']))

for k,v in df.groupby('Gene'):
    print(k)
    print(np.percentile(list(v['SSR']),[50,25,75]))
    
import scipy.stats as stats
al_cds=[];region=[]
for k,v in df.groupby('Gene'):
    al_cds.append(list(v['SSR']))
    region.append(k)
###整体差异
stats.kruskal(al_cds[0],al_cds[1],al_cds[2],al_cds[3],al_cds[4],al_cds[5],al_cds[6],al_cds[7],al_cds[8])


###两两检验
for i in range(9):
    for j in range(i+1,9):
        a,b=stats.mannwhitneyu(al_cds[i],al_cds[j])
        print(region[i],region[j])
        print(a)
        print(b)
        print(b*32)